In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Input, Concatenate
from keras.layers.normalization import BatchNormalization
import keras 
import os 

from keras.applications import VGG16, Xception, NASNetMobile
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from IPython.display import display
from PIL import Image

Using TensorFlow backend.


In [3]:
# Hackish way of doing things 
# !rm -rf Cancer-Cells-Classification
# !git clone https://github.com/lionellloh/Cancer-Cells-Classification.git
# # !wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5

Cloning into 'Cancer-Cells-Classification'...
remote: Enumerating objects: 519, done.
remote: Counting objects: 100% (519/519), done.
remote: Compressing objects: 100% (516/516), done.
remote: Total 519 (delta 5), reused 517 (delta 3), pack-reused 0
Receiving objects: 100% (519/519), 56.35 MiB | 23.21 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [4]:
default_model = VGG16(weights = None, include_top = False)

x = default_model.output 
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
# x = Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# simple_cnn = Sequential()
# simple_cnn.add(BatchNormalization(input_shape = (224, 224, 3),name = 'NormalizeInput'))
# simple_cnn.add(Conv2D(8, kernel_size = (3,3), padding = 'same'))
# simple_cnn.add(Conv2D(8, kernel_size = (3,3), padding = 'same'))
# # use dilations to get a slightly larger field of view
# simple_cnn.add(Conv2D(16, kernel_size = (3,3), dilation_rate = 2, padding = 'same'))
# simple_cnn.add(Conv2D(16, kernel_size = (3,3), dilation_rate = 2, padding = 'same'))
# simple_cnn.add(Conv2D(32, kernel_size = (3,3), dilation_rate = 3, padding = 'same'))
# # the final processing
# simple_cnn.add(Conv2D(16, kernel_size = (1,1), padding = 'same'))
# simple_cnn.add(Conv2D(1, kernel_size = (1,1), padding = 'same', activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model=Model(inputs=default_model.input, outputs = predictions)
print(model.summary())

# simple_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [6]:
for i, layer in enumerate(model.layers):
    if i < 15: 
        layer.trainable = False
        
    else: 
        layer.trainable = True
        
    print(f"Layer {i}, {layer.name}, Trainable: {layer.trainable}")

Layer 0, input_1, Trainable: False
Layer 1, block1_conv1, Trainable: False
Layer 2, block1_conv2, Trainable: False
Layer 3, block1_pool, Trainable: False
Layer 4, block2_conv1, Trainable: False
Layer 5, block2_conv2, Trainable: False
Layer 6, block2_pool, Trainable: False
Layer 7, block3_conv1, Trainable: False
Layer 8, block3_conv2, Trainable: False
Layer 9, block3_conv3, Trainable: False
Layer 10, block3_pool, Trainable: False
Layer 11, block4_conv1, Trainable: False
Layer 12, block4_conv2, Trainable: False
Layer 13, block4_conv3, Trainable: False
Layer 14, block4_pool, Trainable: False
Layer 15, block5_conv1, Trainable: True
Layer 16, block5_conv2, Trainable: True
Layer 17, block5_conv3, Trainable: True
Layer 18, block5_pool, Trainable: True
Layer 19, global_average_pooling2d_1, Trainable: True
Layer 20, dense_1, Trainable: True
Layer 21, dense_2, Trainable: True
Layer 22, dense_3, Trainable: True
Layer 23, dropout_1, Trainable: True
Layer 24, dense_4, Trainable: True
Layer 25, dens

In [7]:
# train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_datagen = ImageDataGenerator(
                  rescale = 1./255,
                  horizontal_flip = True,
                  fill_mode = "nearest",
                  zoom_range = 0.3,
                  width_shift_range = 0.3,
                  height_shift_range=0.3,
                  rotation_range=30)

val_datagen = ImageDataGenerator(
                  rescale = 1./255,
                  horizontal_flip = True,
                  fill_mode = "nearest",
                  zoom_range = 0.3,
                  width_shift_range = 0.3,
                  height_shift_range=0.3,
                  rotation_range=30)


train_generator=train_datagen.flow_from_directory('Cancer-Cells-Classification/train',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=True)

val_datagen= ImageDataGenerator(rescale =1./255)
val_generator = val_datagen.flow_from_directory('Cancer-Cells-Classification/val',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=True)

Found 336 images belonging to 2 classes.
Found 167 images belonging to 2 classes.


In [15]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose = 1, save_best_only=True)
callbacks_list = [mc]
step_size_train=train_generator.n//train_generator.batch_size
step_size_val =val_generator.n//val_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=val_generator,
                    validation_steps=step_size_val,
                    epochs=30, 
                    callbacks=callbacks_list)


Epoch 1/30
10/10 [==============================] - 15s 2s/step - loss: 0.4551 - acc: 0.8063 - val_loss: 0.5591 - val_acc: 0.7125

Epoch 00001: val_acc improved from -inf to 0.71250, saving model to best_model.h5
Epoch 2/30
10/10 [==============================] - 10s 981ms/step - loss: 0.4435 - acc: 0.8153 - val_loss: 0.5810 - val_acc: 0.6815

Epoch 00002: val_acc did not improve from 0.71250
Epoch 3/30
10/10 [==============================] - 11s 1s/step - loss: 0.4796 - acc: 0.8065 - val_loss: 0.5836 - val_acc: 0.7333

Epoch 00003: val_acc improved from 0.71250 to 0.73333, saving model to best_model.h5
Epoch 4/30
10/10 [==============================] - 11s 1s/step - loss: 0.4747 - acc: 0.7654 - val_loss: 0.5920 - val_acc: 0.6815

Epoch 00004: val_acc did not improve from 0.73333
Epoch 5/30
10/10 [==============================] - 11s 1s/step - loss: 0.4269 - acc: 0.7996 - val_loss: 0.7288 - val_acc: 0.5556

Epoch 00005: val_acc did not improve from 0.73333
Epoch 6/30
10/10 [=======

In [16]:
import numpy as np

np.random.seed(7)
print("Loading previous weights")
model.load_weights("best_model.h5")
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Loading previous weights


In [52]:
test_datagen= ImageDataGenerator(rescale =1./255)
test_generator = test_datagen.flow_from_directory('Cancer-Cells-Classification/val',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='binary',
                                                 shuffle=False)

filenames = test_generator.filenames
number_of_samples = len(filenames)

predictions = model.predict_generator(test_generator, steps = 6)
performance = model.evaluate_generator(test_generator,steps = 6)

Found 167 images belonging to 2 classes.


In [53]:
print(model.metrics_names)
# print(predictions)

print(f"{model.metrics_names[0]}: {performance[0]} and {model.metrics_names[1]}: {performance[1]}")

['loss', 'acc']
loss: 0.4590100296243222 and acc: 0.7964071859856565


In [54]:
y_pred = []
for value in predictions: 
  if value >= 0.5: 
    y_pred.append(1)
  else: 
    y_pred.append(0)
    
print(y_pred)

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]


In [58]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

print(classification_report(list(test_generator.classes), y_pred, target_names= ["negative", "positive"]))

              precision    recall  f1-score   support

    negative       0.87      0.79      0.83       103
    positive       0.70      0.81      0.75        64

   micro avg       0.80      0.80      0.80       167
   macro avg       0.79      0.80      0.79       167
weighted avg       0.81      0.80      0.80       167



In [60]:
print(roc_auc_score(list(test_generator.classes), y_pred))

0.7994538834951456
